---

Created for [learn-investments.rice-business.org](https://learn-investments.rice-business.org)
    
By [Kerry Back](https://kerryback.com) and [Kevin Crotty](https://kevin-crotty.com)
    
Jones Graduate School of Business, Rice University

---


# READ DATA

In [54]:
import pandas as pd
from pandas_datareader import DataReader as pdr

# nominal returns on stocks, bonds, bills, and gold
df = pd.read_csv(
    "https://www.dropbox.com/scl/fi/qzdune8v7txf78sqplmvm/sbbg_nominal.csv?rlkey=3eif98d1oy1u3k9husmclcnkb&dl=1",
    index_col="Year"
)

# inflation rate
inflation = pdr("CPIAUCSL", "fred", start="1967-01-01")
inflation = inflation.resample("Y").last().iloc[:-1]
inflation.index = [x.year for x in inflation.index]
inflation = inflation.pct_change().dropna()
df["inflation"] = inflation

# CALCULATE REAL RETURNS

In [55]:
df = (1+df).div(1+df.inflation, axis="index") - 1
df = df.drop(columns=["inflation"])

# SUMMARY STATISTICS

In [56]:
df.describe().iloc[1:]

,S&P 500,Gold,Corporates,Treasuries,TBills
mean,0.071532,0.055711,0.044882,0.024660,0.004179
std,0.168236,0.236037,0.092577,0.105357,0.025348
min,-0.365382,-0.376982,-0.196658,-0.228034,-0.066696
25%,-0.015712,-0.083743,-0.015566,-0.040691,-0.012671
50%,0.106083,0.015780,0.062383,0.006578,0.002431
75%,0.182826,0.171412,0.103233,0.102256,0.020313
max,0.338077,1.060907,0.242972,0.279207,0.065280


# CORRELATION MATRIX

In [57]:
df.corr().round(3)

,S&P 500,Gold,Corporates,Treasuries,TBills
S&P 500,1.000,-0.199,0.546,0.185,0.198
Gold,-0.199,1.000,-0.163,-0.116,-0.360
Corporates,0.546,-0.163,1.000,0.716,0.491
Treasuries,0.185,-0.116,0.716,1.000,0.576
TBills,0.198,-0.360,0.491,0.576,1.000


# FIGURE 1 (BOXPLOTS)

In [58]:
import plotly.graph_objects as go 

traces = []
for asset in df.columns:
    trace = go.Box(
        y=df[asset].to_numpy(),
        name=asset,
        hovertemplate="%{y:.1%}<extra></extra>"
    )
    traces.append(trace)

fig = go.Figure()
for trace in traces:
    fig.add_trace(trace)
    
fig.update_layout(
    yaxis_tickformat=".0%",
    template="plotly_white"
)
fig.show()    


# FIGURE 2 (COMPOUND RETURNS)

In [59]:
accum = (1+df).cumprod()

traces = []
for asset in accum.columns:
    trace = go.Scatter(
        x=accum.index.to_list(),
        y=accum[asset],
        name=asset,
        text=[asset]*accum.shape[0],
        hovertemplate="%{text} = $%{y:.2f}<extra></extra>"
    )
    traces.append(trace)

fig = go.Figure()
for trace in traces:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_tickprefix="$",
    xaxis_tickformat=".0f",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01
    ),
)
fig.show()

# FIGURE 3 (COMPOUND RETURNS ON LOG SCALE)

In [60]:
accum = (1+df).cumprod()

traces = []
for asset in accum.columns:
    trace = go.Scatter(
        x=accum.index.to_list(),
        y=accum[asset],
        name=asset,
        text=[asset]*accum.shape[0],
        hovertemplate="%{text} = $%{y:.2f}<extra></extra>"
    )
    traces.append(trace)

fig = go.Figure()
for trace in traces:
    fig.add_trace(trace)

fig.update_layout(
    yaxis_tickprefix="$",
    xaxis_tickformat=".0f",
    yaxis_type="log",
    hovermode="x unified",
    template="plotly_white",
    legend=dict(
        yanchor="top", 
        y=0.99, 
        xanchor="left", 
        x=0.01
    ),
)
fig.show()